In [7]:
class MSH_parser:
    def __init__(self):
        self.nodes_dict = {}      # Nodes dict <---> node_ID: [x, y, z]
        self.elems_dict = {}      # Elems dict <---> elem_ID: [ne_1, ne_2, ...]
    
    # Reads nodes block from *.msh file
    def read_nodes_block(self, file_stream, N_nodes):
        indicies = []
        coords = []

        # Reading indicies
        for node_id in range(N_nodes):
            node_index = int(file_stream.readline())
            indicies.append(node_index)

        # Reading coordinates
        for node_id in range(N_nodes):
            node_xyz = list(map(float, file_stream.readline().split(' ')))
            coords.append(node_xyz)

        return indicies, coords

    # Reads elems block from *.msh file
    def read_elems_block(self, file_stream, N_elems):
        indicies = []
        nodes = []

        for elem_id in range(N_elems):
            elem_info = file_stream.readline().replace(' \n', '').split(' ')
            indicies.append(int(elem_info[0]))
            nodes.append(list(map(int, elem_info[1:])))

        return indicies, nodes

    # Parses *.msh file
    def parse_file(self, filename):
        print(f'File: \t\t{filename}')
        print('-'*50)

        input_file = open(filename, 'r')
        prev_line = input_file.readline()
        curr_line = None

        for curr_line in input_file:
            if (prev_line.count('$Nodes') != 0):
                break
            prev_line = curr_line

        node_blocks = int(curr_line.split(' ')[0])

        print(f'Node blocks: \t{node_blocks}')

        # Reading Nodes
        for node_block in range(node_blocks):
            block_info = input_file.readline()
            block_size = int(block_info.split(' ')[-1])
            indicies, coords = self.read_nodes_block(input_file, block_size)
            for i, ID in enumerate(indicies):
                self.nodes_dict[ID] = coords[i]

        print(f'Nodes: \t\t{len(self.nodes_dict.keys())}\n')

        # Skipping '$EndNodes' tag
        prev_line = input_file.readline()

        # Skipping '$Elements' tag
        prev_line = input_file.readline()

        # Setting reader
        prev_line = input_file.readline()

        elem_blocks = int(prev_line.split(' ')[0])
        n_elems = int(prev_line.split(' ')[-1])

        print(f'Elem. blocks: \t{elem_blocks}')
        print(f'Elements: \t{n_elems}')

        # Reading Elements    
        for elem_block in range(elem_blocks):
            block_info = input_file.readline()
            block_size = int(block_info.split(' ')[-1])
            block_dim = int(block_info.split(' ')[0])
            indicies, nodes = self.read_elems_block(input_file, block_size)
            if (block_dim < 2):
                continue
            for i, ID in enumerate(indicies):
                self.elems_dict[ID] = nodes[i]

        input_file.close()

        print(f'\nReading done\n{"-"*50}')
        
    def get_mesh(self):
        return self.nodes_dict, self.elems_dict
    
    def save_mesh(self, save_name):
        # Saving nodes data
        nodes_file = open(save_name + '_nodes.txt', 'w')
        
        for key in list(self.nodes_dict.keys()):
            # Writing in format [ID x y z]
            nodes_file.write(f'{key} {self.nodes_dict[key][0]} {self.nodes_dict[key][1]} {self.nodes_dict[key][2]}\n')
            
        nodes_file.close()
        print('Nodes saved')
        
        # Saving elems data
        elems_file = open(save_name + '_elems.txt', 'w')
        
        for key in list(self.elems_dict.keys()):
            # Writing in format [ID ne_1 ne_2 ...]
            write_str = f'{key} '
            for node in self.elems_dict[key]:
                write_str += str(node) + ' '
            elems_file.write(write_str + '\n')
            
        elems_file.close()
        print('Elems saved')

In [8]:
filename_1 = 'Plane_With_Hole_2.msh'

In [9]:
parser = MSH_parser()
parser.parse_file(filename_1)
parser.save_mesh('Mesh_1')

mesh_nodes, mesh_elems = parser.get_mesh()

File: 		Plane_With_Hole_2.msh
--------------------------------------------------
Node blocks: 	11
Nodes: 		221

Elem. blocks: 	11
Elements: 	447

Reading done
--------------------------------------------------
Nodes saved
Elems saved
